![](imgs/miredtwitter_header2.png)  
<center>
<h1>
Dime a quiénes sigues y te diré qué prefieres
</h1>
<h2>
(Análisis de Redes Sociales, Aprendizaje Automático y un poquito de PLN)
</h2>
<h3>Pablo Gabriel Celayes</h3>
<h4>Jueves 16 de noviembre de 2017 - PyData San Luis</h4>
</center>

# Resumen

* Introducción

* Herramientas

* Datos

* Predicción social

* Agregando PLN


# Introducción
## Idea original

- Recomendador *personalizado* de artículos basado en contenido ( NLP )
- Mejorarlo con información social ( de fuentes externas o relaciones *inferidas* ) 
- Preferencias de usuarios de Cogfor

![](imgs/cogfor.png)

## Mutación

- Set the datos propio ( usuarios, preferencias, conexiones ) 
- Predecir preferencias usando información social
- Combinar con recomendación basada en contenido


# Herramientas

## Datos
![](imgs/tweepy.png)
![](imgs/sqlalchemy.png)

## Grafos
![](imgs/networkx.png)
![](imgs/graphtool.png)

## Análisis
![](imgs/numpy.png)
![](imgs/pandas.png)
![](imgs/jupyter.png)

## ML + PLN
![](imgs/sklearn.png)
![](imgs/nltk.png)
![](imgs/gensim.png)
![](imgs/twitterLDA.png)


## Visualización
![](imgs/bokeh.png)
![](imgs/gephi_small.png)

# Datos: Grafo social

- hasta 3 pasos de la relación $\texttt{seguir}$ empezando de mí mismo
- filtramos usuarios **relevantes** ( >40 seguidos/seguidores )
- ~3M usuarios
- ~10M conexiones


***Primer paso...***

( no sería una charla de SNA sin un dibujito de estos )

![](imgs/miredtwitter.png)


# Datos: Contenido

- Subgrafo $G$:
    - partimos de un conjunto pequeño de usuarios
    - agregamos 50 *seguidos* más *afines* ( según *seguidos* en común )
    - repetimos hasta que no hay nuevos
    - 5180 usuarios
    - ~230k conexiones

- Tweets:
    - *timelines* entre 25/8/2015 y 24/9/2015 (+ *retweets* y *favs*)
    - 2M tweets ( 1,6M en castellano )

# Predicción social

¿ Dado un usuario $u$, cuánto puedo saber del **contenido** que **prefiere** sabiendo lo que prefieren los usuarios de su **entorno** social ?

- **Contenido** =  tweets en español _"visibles"_ ( $T_u$ )
- **Preferencias** = retweets
- **Entorno** = seguidos + seguidos-por-seguidos ( $E_u$ )


## Tweets visibles

* Compartidos por $u$ o sus seguidos
* Excluímos generados por $u$

* máximo $10000$ ( submuestra de negativos si es necesario )

## Selección de usuarios

* Entrenar y evaluar modelos es caro
* Soy vago y quería hacer todo en mi laptop
* Así que elegimos a quiénes analizar:
    * $A$ = $1000$ más *activos* (cant. de tweets)
    * $I$ = $1000$ más *importantes* (centralidad de Katz)
    * Tomamos $A \cap I$ : $194$ usuarios


## Extracción de características

* $E_u = \{u_1, u_2, \ldots , u_n \}$ 

* $T_u = \{ t_1, \ldots, t_m \}$

$$
  M_u := [ \verb|tweet_en_tl|(t_i, u_j) ]_{\substack{ 1 \leq i \leq m \\ 1 \leq j \leq n}}  
$$


$$
  y_u := [ \texttt{tweet_en_tl}(t_i, u) ]_{ 1 \leq i \leq m }  
$$



## Problema de clasificación

* Predecir $y_u$ en base a filas de $M_u$
* Particionado:
    * $70\%$ entrenamiento ($M^{en}_u, y^{en}_u$)
    * $10\%$ ajuste ($M^{aj}_u, y^{aj}_u$)
    * $20\%$ evaluación ($M^{ev}_u, y^{ev}_u$)

## Support Vector Machines

* Objetivo: **maximizar** margen y **minimizar** errores
* Funciones *kernel* permiten encontrar fronteras **no lineales**:
    * *Radial Basis Function*
    * Polinómico

![](imgs/svm_linsep_err.png)



## Calidad de clasificación

$$\texttt{precision} := \frac{|\{x_i | f(x_i) = 1 \text{ y } y_i = 1 \}|}{|\{x_i | f(x_i) = 1 \}|}$$

$$ $$

$$\texttt{recall} := \frac{|\{x_i | f(x_i) = 1 \text{ y } y_i = 1 \}|}{|\{x_i | y_i = 1 \}|}$$

$$ $$

$$\texttt{F1} := \frac{2}{\frac{1}{\texttt{precision}} + \frac{1}{\texttt{recall}}} = \frac{2 * \texttt{precision} * \texttt{recall} }{\texttt{precision} + \texttt{recall}}$$



## Ajuste de parámetros

* Búsqueda exhaustiva $\verb|GridSearchCV|$
* Validación cruzada en $3$ partes
* Objetivo: maximizar $F1$
* Grilla:
```
{
    "C": [ 0.01, 0.1, 1 ],
    "class_weight": [ "balanced", None ], 
    "gamma": [ 0.1, 1, 10 ],
    "kernel": [ "rbf", "poly" ]
}
```
    * $C$: controla balance entre margen y errores
    * $class\_weight$: ¿dar más importancia a clase minoritaria?
    * $gamma$: forma de la frontera de decisión

## Resultados

* $F1$ sobre $M^{ev}_u$
* Promedio $87,7 \%$

![](imgs/f1s_social_valid.png)

# *¡Y sin tener en cuenta el contenido!*



![](imgs/robot2.png)

# Agregando PLN...

![](imgs/robotlee.jpg)

## Selección de usuarios

* $F1 < 0,75$ en $M^{aj}_u$ ( $27$ usuarios )

* $10$ usuarios más (al azar)

## Pre-procesamiento

- Normalización
- Tokenizado
- Diccionario
- *Bag of words*
- LDA


## Normalización
- quitar URLs
- minúscula
- quitar acentos
- contraer repeticiones de vocales y espacios

![](imgs/normalize.png)

## Tokenizado
- partir en palabras
- remover puntuación
- _stemming_
- remover palabras de 1 caracter

![](imgs/tokenize.png)

## Diccionario

* Vocabulario: todo $T$ tokenizado.
* significativas ( en al menos $100$ tweets ).
* informativas ( en a lo sumo el $30\%$ de los tweets ).

* Diccionario $D$ de ~11K términos.



## *Bag of words*

- Texto $t$:  $\rightarrow$ multiconjunto (*bag*) de las palabras en $t$.
- No importa el orden, pero sí repeticiones.
- En tweets ( $\leq 140$ caracteres ), en general son *conjuntos* ( $0$ o $1$ ocurrencia).

- Diccionario fijo $D = \{ t_1, \ldots, t_{11000} \}$ : $\rightarrow$ vector de características enteras (booleanas):

$$ v_{BOT}(tweet) = [count(t_i, tokens(tweet))]_{i=1}^{11000} $$

- $v_{BOT}(tweet)$ se representa ralo (*sparse*)


## LDA

* Descubre temas subyacentes en textos

* Reducción de dimensionalidad ( espacio de *palabras* en $D$ a espacio de *temas* )


### Ejemplo: LDA 10 temas

![](imgs/lda10.png)

## Twitter-LDA

- Problema: tweets son cortos y normalmente hablan de un único tema 

- Agrupa por usuario
- Asigna un tema por tweet

## Evaluando sobre $M_u^{ev}$

![](imgs/f1s_social_vs_socialldas.png)


* Mejor: $\texttt{TwitterLDA10}$ ( mejora media de $1,7\%$)
* Por ahora agregar NLP no mejora mucho
* Muchos casos mejoran en *train* pero decaen en *test* (*overfitting*)

# ¿Cómo sigue?

- Considerar temporalidad
- Características adicionales
- Generalizar ( modelo que no dependa del usuario )
- _Retweeteabilidad_ en comunidades

- Resolver overfitting en caso Social+NLP
- Probar word embeddings en la parte NLP?
